In [8]:
import pyspark
from pyspark.sql import SparkSession
import random

In [9]:
sp = SparkSession.builder.config('spark.driver.memory', '4g').appName('square_integers').getOrCreate()
nums = random.sample(range(0, 100), 10)
df_squared = sp.createDataFrame([(n**2,) for n in nums], ['numbers'])
df_squared.show()
nums

+-------+
|numbers|
+-------+
|    900|
|   1089|
|   5476|
|   1600|
|    289|
|   5184|
|   2916|
|   2025|
|   8464|
|     64|
+-------+



[30, 33, 74, 40, 17, 72, 54, 45, 92, 8]

In [20]:
from pyspark.sql.functions import max, avg
max_num = df_squared.agg(max('numbers')).collect()[0][0]
max_num

8464

In [17]:
meany = df_squared.agg(avg('numbers')).collect()[0][0]
meany

2800.7

In [21]:

df = sp.read.csv('colors.csv', header = True, inferSchema = True)
df.show()
df.printSchema()

+-------+------+-----+
|Numbers| Color|Price|
+-------+------+-----+
|      1|   Red|  500|
|      2|Yellow|  500|
|      3|  Blue| 1200|
|      4| Green|  600|
|      5|  Aqua| 1400|
|      6| White|  200|
+-------+------+-----+

root
 |-- Numbers: integer (nullable = true)
 |-- Color: string (nullable = true)
 |-- Price: integer (nullable = true)



In [23]:
price = df.select("price").summary("count", "min", "max", "mean", "stddev")
price.show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|                6|
|    min|              200|
|    max|             1400|
|   mean|733.3333333333334|
| stddev|463.3213427705081|
+-------+-----------------+



In [36]:
from pyspark.sql import functions as F

split_on_spaces = F.split('value', ' ')
df = (
    sp.read.option("recursiveFileLookup","true").text('TextDocs/')
    .withColumn('value', F.explode(split_on_spaces))
    .groupBy('value').count()
    .orderBy(F.desc('count'))
)
top_val_dict = {r['value']:r['count'] for r in df.head(10)}

print(top_val_dict)

{'': 81, 'the': 59, 'was': 41, 'to': 39, 'and': 33, 'he': 28, 'that': 28, 'a': 28, 'of': 20, 'her': 20}
